In [26]:
import os
from dotenv import load_dotenv

In [27]:
load_dotenv()

True

In [28]:
groq_api_key = os.getenv('GROQ_API_KEY')

gemini_api_key = os.getenv('GEMINI_API_KEY')

#### *Extraction Based Approach*

In [29]:
from langchain.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
import re

def load_and_extract(path):
    loader = PyPDFLoader(path,extraction_mode='plain',extraction_kwargs={'codec':'utf-8'})
    docs = loader.load()

    text = " ".join([doc.page_content for doc in docs])

    text = re.sub(r'\s+',' ',text)

    splitter = RecursiveCharacterTextSplitter(chunk_size=200,chunk_overlap=50)

    docs = splitter.split_text(text)

    return docs

In [30]:
def preprocess_response(response):
    response = re.sub(r"<think>.*?</think>","",response,flags=re.DOTALL)

    return response

In [31]:
from langchain_groq import ChatGroq

deepseek = ChatGroq(model="deepseek-r1-distill-qwen-32b",temperature=0.6,api_key=groq_api_key)

In [32]:
qwen = ChatGroq(model="qwen-qwq-32b",temperature=0.6,api_key=groq_api_key)

In [33]:
from langchain_core.prompts import ChatPromptTemplate

template = '''
You are a seasoned startup advisor with over 15 years of experience evaluating pitch decks for various industries. 
Your expertise lies in analyzing the components of a pitch deck to provide actionable feedback, scoring, and insights on strengths and weaknesses.
Your task is to evaluate a startup's pitch deck and provide a pitch score based on these criteria:
- Problem Statement (0-15)
- Solution Clarity (0-15)
- Market Opportunity (0-15)
- Business Model (0-10)
- Competitive Advantage (0-10)
- Team Strength (0-10)
- Traction & Metrics (0-10)
- Financial projections (0-10)
- Investment Ask (0-5)

pitch deck : {doc}

Provide a score for each section and also provide the total score out of hundred by adding all the score of each section.

Example output :

    Problem Statement = 10
    Solution Clarity = 12
    Market Opportunity = 10
    Business Model = 10
    Competitive Advantage = 10
    Team Strength = 5
    Traction & Metrics = 8
    Financial projections = 10
    Investment Ask = 5

    Total pitch_score = 80/100

Make sure don't provide any information other than the output stated in example.
'''

structured_prompt = ChatPromptTemplate.from_template(
    template=template
)

In [34]:
deepseek_chain = structured_prompt|deepseek

qwen_chain = structured_prompt|qwen

#### *OCR using Vision model*

In [1]:
from pdf2image import pdf2image

def PDF2IMG(file_path,fmt='ppm'):
    '''
    Args : 
        file_path : The path of the pdf file of pitch, make sure to provide the path as raw string.
        fmt : The format of images as output
    Output :
        list of images
    '''
    
    images = pdf2image.convert_from_path(file_path,fmt=fmt)

    return images

In [7]:
vision_model = ChatGroq(model='llama-3.2-90b-vision-preview',api_key=groq_api_key)

In [10]:
vision_template = """
You are a seasoned startup advisor with over 15 years of experience evaluating pitch decks for various industries. 
Your expertise lies in analyzing the components of a pitch deck to provide actionable feedback, scoring, and insights on strengths and weaknesses.
Your task is to evaluate a startup's pitch deck and provide a pitch score based on these criteria:
- Problem Statement (0-15)
- Solution Clarity (0-15)
- Market Opportunity (0-15)
- Business Model (0-10)
- Competitive Advantage (0-10)
- Team Strength (0-10)
- Traction & Metrics (0-10)
- Financial projections (0-10)
- Investment Ask (0-5)

pitch deck : {images}

Provide a score for each section and also provide the total score out of hundred by adding all the score of each section.

Example output :

    Problem Statement = 10
    Solution Clarity = 12
    Market Opportunity = 10
    Business Model = 10
    Competitive Advantage = 10
    Team Strength = 5
    Traction & Metrics = 8
    Financial projections = 10
    Investment Ask = 5

    Total pitch_score = 80/100

Make sure don't provide any information other than the output stated in example.
"""

vision_prompt = ChatPromptTemplate.from_template(
    template=vision_template
)

In [11]:
vision_chain = vision_prompt|vision_model

## ***Evaluation***

In [17]:
import google.generativeai as genai

genai.configure(api_key=gemini_api_key)

In [35]:
for model in genai.list_models():
    if 'generateContent' in model.supported_generation_methods:
        print(model.name)

models/gemini-1.0-pro-vision-latest
models/gemini-pro-vision
models/gemini-1.5-pro-latest
models/gemini-1.5-pro-001
models/gemini-1.5-pro-002
models/gemini-1.5-pro
models/gemini-1.5-flash-latest
models/gemini-1.5-flash-001
models/gemini-1.5-flash-001-tuning
models/gemini-1.5-flash
models/gemini-1.5-flash-002
models/gemini-1.5-flash-8b
models/gemini-1.5-flash-8b-001
models/gemini-1.5-flash-8b-latest
models/gemini-1.5-flash-8b-exp-0827
models/gemini-1.5-flash-8b-exp-0924
models/gemini-2.5-pro-exp-03-25
models/gemini-2.5-pro-preview-03-25
models/gemini-2.0-flash-exp
models/gemini-2.0-flash
models/gemini-2.0-flash-001
models/gemini-2.0-flash-exp-image-generation
models/gemini-2.0-flash-lite-001
models/gemini-2.0-flash-lite
models/gemini-2.0-flash-lite-preview-02-05
models/gemini-2.0-flash-lite-preview
models/gemini-2.0-pro-exp
models/gemini-2.0-pro-exp-02-05
models/gemini-exp-1206
models/gemini-2.0-flash-thinking-exp-01-21
models/gemini-2.0-flash-thinking-exp
models/gemini-2.0-flash-thinki

In [18]:
gemini_model = genai.GenerativeModel(model_name="gemini-2.0-flash")

#### *Evaluation for Extraction Approach*

In [36]:
pitch = load_and_extract(r"C:\Users\SHIVAM GHUGE\Downloads\AirBedPitch.pdf")

In [37]:
qwen_pitch = qwen_chain.invoke({'doc':pitch})

qwen_pitch_score = preprocess_response(qwen_pitch.content).strip()

In [41]:
print(qwen_pitch_score)

Problem Statement = 14  
Solution Clarity = 13  
Market Opportunity = 12  
Business Model = 8  
Competitive Advantage = 3  
Team Strength = 0  
Traction & Metrics = 2  
Financial projections = 5  
Investment Ask = 0  

Total pitch_score = 57/100


In [38]:
deepseek_response = deepseek_chain.invoke({'doc':pitch})

deepseek_pitch_score = preprocess_response(deepseek_response.content).strip()

In [42]:
print(deepseek_pitch_score)

Problem Statement = 14
    Solution Clarity = 13
    Market Opportunity = 13
    Business Model = 9
    Competitive Advantage = 7
    Team Strength = 0
    Traction & Metrics = 8
    Financial projections = 0
    Investment Ask = 0

    Total pitch_score = 64/100


In [43]:
evaluation_prompt = f'''
You are an AI evaluator. Two LLMs (deepseek-r1 and qwen-32b) have analyzed a startup pitch deck and provided pitch scores.
deepseek pitch score : {deepseek_pitch_score}
qwen pitch score : {qwen_pitch_score}
Your task:
    - Review both of their section scores and Total pitch_score.
    - Identify areas where they agree/disagree based on section score.
    - Provide personalized feedback on which areas the startup pitch need improvement and suggest content improvements or additional data needed.
    - Provide the final total score for the pitch based on {deepseek_pitch_score} and {qwen_pitch_score}.

Example output :
    Final Score = 70/100

    personalized feedback : 

Make sure don't provide any information other than the output stated in example and feedback should be under 300 words.
'''

In [44]:
response = gemini_model.generate_content(
    contents=evaluation_prompt
)

In [ ]:
print(response.text)

Final Score = 64/100

personalized feedback :

Both models agree on the weakness of the pitch regarding Team Strength, Financial Projections, and Investment Ask. The models also largely agree on the Problem Statement, Solution Clarity, and Business Model. They disagree most on Competitive Advantage and Traction & Metrics, with qwen-32b scoring these sections lower.

The pitch needs significant improvement in several key areas.

*   **Team Strength:** This is a critical area. Introduce the core team members and highlight their relevant experience and expertise. Explain why this team is uniquely positioned to solve the problem.

*   **Financial Projections:** Include realistic financial projections, even if high-level. Investors need to see the potential for return. Include key assumptions driving the projections.

*   **Investment Ask:** Clearly state the amount of funding sought and how it will be used. Break down the allocation of funds for specific purposes (e.g., product development

***Pitch-II***

#### *Evaluation for OCR Approach*

***Pitch-I***

In [ ]:
response = vision_chain.invoke({
    'images':PDF2IMG(r"C:\Users\SHIVAM GHUGE\Downloads\AirBedPitch.pdf")
})

In [ ]:
pitch_score = response.content

In [23]:
evaluation_prompt_ocr = f'''
    You are an AI evaluator. Vision model have analyzed a startup pitch deck and provided pitch scores.
    pitch score : {pitch_score}
    Your task:
        - Review both of their section scores and Total pitch_score.
        - Identify areas where they agree/disagree based on section score.
        - Provide personalized feedback on which areas the startup pitch need improvement and suggest content improvements or additional data needed.
        - Provide the final total score for the pitch based on {pitch_score}.

    Example output :
        Final Score = 70/100

        personalized feedback : 

    Make sure don't provide any information other than the output stated in example and feedback should be under 300 words.
'''

In [46]:
res = gemini_model.generate_content(
    contents=evaluation_prompt_ocr
)

In [47]:
print(res.text)

Final Score = 71/100

Personalized Feedback:

The pitch demonstrates strength in Solution Clarity (10) and Market Opportunity (12), indicating a clear understanding of the product and its potential. Problem Statement, Business Model, Competitive Advantage, and Financial Projections are adequately addressed (all scoring 8).

However, there's significant room for improvement in Team Strength (6) and Investment Ask (4). The pitch needs to showcase the team's relevant experience and expertise more effectively. Consider adding specific accomplishments, industry connections, or advisory board members. The Investment Ask score suggests this section is weak. Clarify the use of funds, milestones to be achieved, and the expected return for investors. A low score in this section could deter potential investors.

Traction & Metrics (7) could also be strengthened. Provide more concrete data on user acquisition, engagement, or revenue growth to demonstrate early success and validate the business mod

***Pitch-II***